In [12]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix, roc_curve, accuracy_score, f1_score, recall_score, precision_score
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

#from keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras import Model, Sequential
from tensorflow.keras.layers import Dense, Input 
import tensorflow as tf
from scikeras.wrappers import KerasClassifier
import keras

import time

#add other imports here if any (for example, pytorch)
import torch
import torch.nn as nn
import torch.optim as optim

Label_encoder = LabelEncoder()

# Load the dataset
train_data = pd.read_csv('fraudTrain.csv')
pd.set_option('display.max_columns', None)

#Convert Gender
gender_conversion = {'F': 0, 'M': 1}
train_data['gender'] = train_data['gender'].map(gender_conversion)

train_data['name'] = train_data['first'] + " " + train_data['last']
train_data['address'] = train_data['street'] + ", " + train_data['city'] + ", " + train_data['state']

#Convert first name
train_data['name'] = Label_encoder.fit_transform(train_data['name'])

train_data['age'] = 2024 - pd.to_numeric(train_data['dob'].str[:4]) 

#Convert Job
train_data['job'] = Label_encoder.fit_transform(train_data['job'])

train_data['merchant'] = Label_encoder.fit_transform(train_data['merchant'])

train_data['category'] = Label_encoder.fit_transform(train_data['category'])

train_data['address'] = Label_encoder.fit_transform(train_data['address'])

train_data['trans_num'] = Label_encoder.fit_transform(train_data['trans_num'])

train_data = train_data.drop(columns=['first', 'last', 'street', 'street', 'city', 'state', 'trans_date_trans_time', 'Unnamed: 0', 'dob'])


test_data = pd.read_csv('fraudTest.csv')
pd.set_option('display.max_columns', None)

#Convert Gender
gender_conversion = {'F': 0, 'M': 1}
test_data['gender'] = test_data['gender'].map(gender_conversion)

test_data['name'] = test_data['first'] + " " + test_data['last']
test_data['address'] = test_data['street'] + ", " + test_data['city'] + ", " + test_data['state']

#Convert first name
test_data['name'] = Label_encoder.fit_transform(test_data['name'])

test_data['age'] = 2024 - pd.to_numeric(test_data['dob'].str[:4]) 

#Convert Job
test_data['job'] = Label_encoder.fit_transform(test_data['job'])

test_data['merchant'] = Label_encoder.fit_transform(test_data['merchant'])

test_data['category'] = Label_encoder.fit_transform(test_data['category'])

test_data['address'] = Label_encoder.fit_transform(test_data['address'])

test_data['trans_num'] = Label_encoder.fit_transform(test_data['trans_num'])

test_data = test_data.drop(columns=['first', 'last', 'street', 'street', 'city', 'state', 'trans_date_trans_time', 'Unnamed: 0', 'dob'])


train_data.head()
y_train = train_data["is_fraud"]
x_train = train_data.drop(columns=["is_fraud"], axis=1)

In [13]:
test_data.head()
y_test = test_data["is_fraud"]
x_test = test_data.drop(columns=["is_fraud"], axis=1)

In [18]:
sc = StandardScaler() 
x_train_std = sc.fit_transform(x_train)
x_test_std = sc.transform(x_test)

In [19]:
def Grid_Search_NN_model(hidden_neurons=8, learning_rate=0.1):
    # Define the input layer
    input_layer = Input(shape=(x_train_std.shape[1],))  # Input shape is based on your training data shape
    # Define the first hidden layer and the previously connected layer
    hidden1 = Dense(hidden_neurons)(input_layer)
    hidden1 = LeakyReLU(alpha=0.01)(hidden1)# LEAKY RELU!!! super awesome activation function
    # Define the output layer and the previously connected layer
    output = Dense(1, activation='sigmoid')(hidden1)
    
    # Create the model object
    myGSModel = Model(inputs=input_layer, outputs=output)  # Use `inputs` and `outputs` arguments

    # Define optimizer with learning rate
    optimizer =  keras.optimizers.SGD(learning_rate=learning_rate)

    # Compile the model
    myGSModel.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

    return myGSModel

In [ ]:
# Run gridsearch here
param_grid = {
    "model__hidden_neurons": [8, 16, 32],  # Number of neurons in each hidden layer
    "model__learning_rate": [0.01, 0.1],  # Learning rates to test
    "batch_size": [32],                   # Fixed batch size
    "epochs": [20]                        # Number of epochs per training session
}


#to use a keras model with sklearn we need to call a wrapper function where the build function is our previously defined function
model = KerasClassifier(model=Grid_Search_NN_model, verbose=0)
#instantiate gridsearch object using 3 fold crossvaliadtion
grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, scoring='accuracy', verbose=1)
#fit the gridsearch object on the data
grid_result = grid.fit(x_train_std, y_train)
#determine the best parameter
print(grid_result.best_params_)

Fitting 3 folds for each of 6 candidates, totalling 18 fits


In [ ]:

print(f"Best Hyperparameters: {grid_result.best_params_}")
print(f"Best Cross-Validation Accuracy: {grid_result.best_score_:.4f}")


In [ ]:
#BASED OFF OF AWESOME GRID SEARCH RESULTS



In [ ]:
#NEED TO EDIT THIS BASED OFF HYPER PARAMETER OPTIMIZATION


model = Sequential()
model.add(Dense(1, input_shape=(X_train_std.shape[1],), activation = 'tanh'))
model.compile(optimizer = keras.optimizers.SGD(learning_rate=0.1), loss='BinaryCrossentropy', metrics=['accuracy', 'precision'])
print(model.summary())

#maybe add cross validation
history = model.fit(X_train_std, y_train, epochs = 10, verbose=0, batch_size = 32)
losses2 = history.history['loss']
#idk
y_pred = model.predict(X_test_std)
y_pred_val = (y_pred >= 0.5)

plt.figure(figsize=(8, 6))
plt.plot(range(1, len(losses2) + 1), losses2, marker='o', linestyle='-', color='b')
plt.title('Loss vs. Epochs', fontsize=16)
plt.xlabel('Epoch', fontsize=14)
plt.ylabel('Binary Cross-Entropy Loss', fontsize=14)
plt.grid(True)
plt.show()